In [ ]:

!pip install ultralytics -q

import torch
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("Warning: Using CPU")

from google.colab import drive
drive.mount('/content/drive')


In [ ]:

!unzip "/content/drive/MyDrive/volley_player_dataset.zip" -d /content/player_dataset

!cat /content/player_dataset/data.yaml

!yolo task=detect mode=train model=yolov8n.pt \
  data=/content/player_dataset/data.yaml \
  epochs=20 imgsz=640

!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt \
  source=/content/player_dataset/valid/images save=True


In [ ]:

!unzip "/content/drive/MyDrive/volley_ball_dataset.zip" -d /content/ball_dataset

!cat /content/ball_dataset/data.yaml

!yolo task=detect mode=train model=yolov8n.pt \
  data=/content/ball_dataset/data.yaml \
  epochs=20 imgsz=640

!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt \
  source=/content/ball_dataset/valid/images save=True


In [ ]:

!unzip "/content/drive/MyDrive/volley_actions_dataset.zip" -d /content/actions_dataset

import os
from pathlib import Path
import shutil

def filter_spike_only(source_split):
    label_dir = Path(f"/content/actions_dataset/{source_split}/labels")
    image_dir = Path(f"/content/actions_dataset/{source_split}/images")

    target_label_dir = Path(f"/content/spike_dataset/{source_split}/labels")
    target_image_dir = Path(f"/content/spike_dataset/{source_split}/images")
    target_label_dir.mkdir(parents=True, exist_ok=True)
    target_image_dir.mkdir(parents=True, exist_ok=True)

    for label_file in label_dir.glob("*.txt"):
        with open(label_file, "r") as f:
            lines = f.readlines()
        spike_lines = [line for line in lines if line.startswith("4 ")]  # 4 = spike class
        if spike_lines:
            with open(target_label_dir / label_file.name, "w") as f_out:
                f_out.writelines(spike_lines)

            img_path = image_dir / label_file.with_suffix(".jpg").name
            if img_path.exists():
                shutil.copy(img_path, target_image_dir / img_path.name)

filter_spike_only("train")
filter_spike_only("valid")

with open("/content/spike_dataset/data.yaml", "w") as f:
    f.write("""train: /content/spike_dataset/train/images
val: /content/spike_dataset/valid/images
nc: 1
names: ["spike"]
""")

!yolo task=detect mode=train model=yolov8n.pt \
  data=/content/spike_dataset/data.yaml \
  epochs=20 imgsz=640


In [ ]:
from IPython.display import Image, display
import glob

for img_path in sorted(glob.glob("runs/detect/predict/*.jpg"))[:5]:
    display(Image(filename=img_path))
